# Balance Analysis

This notebook analyzes game balance by running simulations between different AI agents and visualizing the results.

## Setup

In [ ]:
import sys
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Import analysis tools
from src.analysis import (
    BalanceAnalyzer,
    BalanceConfig,
    quick_balance_check,
    create_matchup_matrix,
)
from src.players import (
    RandomPlayer,
    GreedyPlayer,
    HeuristicPlayer,
    MCTSPlayer,
)
from src.simulation import MatchRunner

print("Imports successful!")

## Define Player Factories

Create factory functions for each player type we want to test.

In [ ]:
# Player factory functions
player_factories = [
    lambda pid: RandomPlayer(player_id=pid, seed=pid * 1000),
    lambda pid: GreedyPlayer(player_id=pid),
    lambda pid: HeuristicPlayer(player_id=pid),
]

# Labels for plotting
player_labels = ["Random", "Greedy", "Heuristic"]

print(f"Testing {len(player_factories)} player types: {player_labels}")

## Quick Balance Check

Run a quick analysis with fewer games to get an initial overview.

In [ ]:
# Quick check with 20 games per matchup
print("Running quick balance check...")
report = quick_balance_check(player_factories, games_per_matchup=20, seed=42)
print(report.summary())

## Full Balance Analysis

Run a more comprehensive analysis with more games for statistical significance.

In [ ]:
# Configure the analyzer
config = BalanceConfig(
    games_per_matchup=50,  # Increase for more accurate results
    dominance_threshold=0.65,  # Warn if any strategy wins > 65%
    min_win_rate_threshold=0.35,  # Warn if any strategy wins < 35%
    base_seed=42,
)

analyzer = BalanceAnalyzer(config)
print("Running full balance analysis...")
full_report = analyzer.run_analysis(player_factories, verbose=True)
print("\n" + full_report.summary())

## Win Rate Visualization

In [ ]:
try:
    import matplotlib.pyplot as plt

    # Create bar chart of win rates
    fig, ax = plt.subplots(figsize=(10, 6))

    types = list(full_report.win_rates.keys())
    rates = [full_report.win_rates[t] for t in types]

    bars = ax.bar(
        types, rates, color=["#3498db", "#2ecc71", "#e74c3c", "#9b59b6"][: len(types)]
    )
    ax.axhline(y=0.5, color="gray", linestyle="--", label="50% baseline")
    ax.axhline(
        y=config.dominance_threshold,
        color="red",
        linestyle=":",
        alpha=0.5,
        label="Dominance threshold",
    )

    ax.set_ylabel("Win Rate")
    ax.set_title("Overall Win Rates by Player Type")
    ax.set_ylim(0, 1)
    ax.legend()

    # Add value labels on bars
    for bar, rate in zip(bars, rates):
        ax.text(
            bar.get_x() + bar.get_width() / 2,
            bar.get_height() + 0.02,
            f"{rate:.1%}",
            ha="center",
            va="bottom",
            fontsize=12,
        )

    plt.tight_layout()
    plt.show()
except ImportError:
    print("matplotlib not installed - skipping visualization")
    print("Install with: uv add matplotlib")

## Head-to-Head Matchup Analysis

In [ ]:
# Print detailed matchup statistics
print("Head-to-Head Matchup Details:")
print("=" * 60)

for (type1, type2), stats in full_report.matchup_results.items():
    print(f"\n{type1} vs {type2}:")
    print(f"  Games: {stats.games}")
    print(f"  {type1} wins: {stats.wins_1} ({stats.win_rate_1:.1%})")
    print(f"  {type2} wins: {stats.wins_2} ({stats.win_rate_2:.1%})")
    print(f"  Draws: {stats.draws} ({stats.draw_rate:.1%})")
    print(f"  Avg game length: {stats.avg_length:.1f} turns")

## Statistical Significance Testing

In [ ]:
from src.analysis import analyze_matchup

print("Statistical Analysis of Matchups:")
print("=" * 60)

for (type1, type2), stats in full_report.matchup_results.items():
    statistical = analyze_matchup(stats)
    print(f"\n{statistical.summary()}")
    print("-" * 40)

## Game Length Distribution

In [ ]:
print(f"\nOverall Statistics:")
print(f"  Total games: {full_report.total_games}")
print(f"  Average game length: {full_report.avg_game_length:.1f} turns")

if full_report.dominance_warnings:
    print(f"\nBalance Warnings:")
    for warning in full_report.dominance_warnings:
        print(f"  - {warning}")
else:
    print("\nNo balance warnings - all strategies appear viable!")

## Conclusions

Based on the analysis above, you can draw conclusions about:

1. **Strategy Dominance**: Is any strategy significantly stronger than others?
2. **Rock-Paper-Scissors**: Are there interesting counter relationships?
3. **Game Length**: Do certain matchups lead to longer/shorter games?
4. **Draw Rate**: Are draws common (possible stalemate issues)?

Use these insights to:
- Tune evaluation weights in heuristic players
- Adjust game mechanics for better balance
- Identify which cards or strategies need adjustment